## Hardward Check

In [1]:
!head /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0


In [2]:
!head -n 3 /proc/meminfo

MemTotal:       26696412 kB
MemFree:        23341436 kB
MemAvailable:   25289424 kB


In [3]:
!nvidia-smi

Thu Jan 27 12:29:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import 

In [4]:
import ast
import os
import json
import pandas as pd
import torch
import importlib
import cv2
import warnings
import numpy as np

from shutil import copyfile
from tqdm.notebook import tqdm
tqdm.pandas()
from PIL import Image
from string import Template
from IPython.display import display
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from matplotlib import animation, rc

warnings.filterwarnings('ignore')

%matplotlib inline

In [5]:
BASE_PATH='/content/drive/MyDrive/Colab/Great-Barrier-Reef/data/cross-validation'

train_df = pd.read_csv(os.path.join(BASE_PATH, 'train-5folds-v3.csv'))
#train_df = pd.read_csv(os.path.join(BASE_PATH, 'train-10folds-v3.csv'))
FOLD = 2

#train_df = pd.read_csv('/content/drive/MyDrive/Colab/Great-Barrier-Reef/data/train-validation-split/train-0.2.csv')
train_df['old_image_path'] = '/content/train_images/' + "video_" + train_df['video_id'].astype(str) + "/" + train_df['video_frame'].astype(str) + ".jpg"

In [6]:
# check Torch and Cuda version
print(f'Torch: {torch.__version__}')
!nvcc --version

Torch: 1.10.0+cu111
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [7]:
%%time
zip_path = '/content/drive/MyDrive/Colab/Great-Barrier-Reef/data/tensorflow-great-barrier-reef.zip'
!cp '{zip_path}' ./
!unzip -q tensorflow-great-barrier-reef.zip
!rm tensorflow-great-barrier-reef.zip
!ls

drive			       example_test.npy  sample_data  train.csv
example_sample_submission.csv  greatbarrierreef  test.csv     train_images
CPU times: user 2.96 s, sys: 482 ms, total: 3.44 s
Wall time: 8min 20s


## 1. Install YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5

%cd yolov5
!pip install -r requirements.txt

In [ ]:
%cd /content

/content


## Copy YOLOv5

In [8]:
!cp -r /content/drive/MyDrive/Colab/Great-Barrier-Reef/yolov5 /content/yolov5

%cd yolov5
!pip install -r requirements.txt
%cd /content

/content/yolov5
     |████████████████████████████████| 596 kB 5.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
/content


## 2. Prepare COTS dataset for YOLOv5
### A. Prepare dataset and annotations

In [9]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

# Annotations
train_df['annotations'] = train_df['annotations'].progress_apply(lambda x: ast.literal_eval(x))
train_df['bboxes'] = train_df.annotations.progress_apply(get_bbox)

#Images resolution
train_df["width"] = 1280
train_df["height"] = 720

  0%|          | 0/23501 [00:00<?, ?it/s]

  0%|          | 0/23501 [00:00<?, ?it/s]

### Helper Function

In [10]:
def voc2yolo(image_height, image_width, bboxes):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    return bboxes

def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

def coco2yolo(image_height, image_width, bboxes):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

def yolo2coco(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes


def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)


def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2):  
     
    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors
    
    if bbox_format == 'yolo':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:
            
                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2 
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)
            
    elif bbox_format == 'coco':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:            
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)

    elif bbox_format == 'voc_pascal':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes: 
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row


# https://www.kaggle.com/diegoalejogm/great-barrier-reefs-eda-with-animations
def create_animation(ims):
    fig = plt.figure(figsize=(16, 12))
    plt.axis('off')
    im = plt.imshow(ims[0])

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//12)

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

### Make directory

In [11]:
HOME_DIR = '/content/'
IMAGE_DIR = 'dataset/images'
LABEL_DIR = 'dataset/labels'

!mkdir {HOME_DIR}dataset
!mkdir {HOME_DIR}{IMAGE_DIR}
!mkdir {HOME_DIR}{LABEL_DIR}

### Extra. Remove unlabeled data

In [12]:
#print(train_df.is_train.value_counts())
print(train_df.fold.value_counts())
train_df.drop(train_df[train_df['n_annotations']==0].index, inplace=True)
#print(train_df.is_train.value_counts())
print(train_df.fold.value_counts())
train_df.reset_index(drop=True, inplace=True)

4    7680
0    5223
1    4030
3    3969
2    2599
Name: fold, dtype: int64
3    1540
1     884
2     876
0     846
4     773
Name: fold, dtype: int64


### Get Paths

In [13]:
def get_path(row):
    
    row['image_path'] = f'{HOME_DIR}{IMAGE_DIR}/{row.image_id}.jpg'
    row['label_path'] = f'{HOME_DIR}{LABEL_DIR}/{row.image_id}.txt'

    return row

train_df = train_df.progress_apply(get_path, axis=1)

  0%|          | 0/4919 [00:00<?, ?it/s]

### Copy 

In [14]:
%%time

for i in tqdm(range(len(train_df))):
    row = train_df.loc[i]
    copyfile(f'{row.old_image_path}', f'{HOME_DIR}{IMAGE_DIR}/{row.image_id}.jpg')
    
print(f'Number of files: {len(os.listdir(f"{HOME_DIR}{IMAGE_DIR}/"))}')

  0%|          | 0/4919 [00:00<?, ?it/s]

Number of files: 4919
CPU times: user 2.99 s, sys: 4.63 s, total: 7.63 s
Wall time: 24.8 s


### Create labels

In [15]:
cnt = 0
all_bboxes = []

for row_idx in tqdm(range(train_df.shape[0])):
    row = train_df.iloc[row_idx]
    image_height = row.height
    image_width  = row.width
    bboxes_coco  = np.array(row.bboxes).astype(np.float32).copy()
    num_bbox     = len(bboxes_coco)
    names        = ['cots']*num_bbox
    labels       = [0]*num_bbox
    ## Create Annotation(YOLO)
    with open(row.label_path, 'w') as f:
        if num_bbox<1:
            annot = ''
            f.write(annot)
            cnt+=1
            continue
        bboxes_yolo  = coco2yolo(image_height, image_width, bboxes_coco)
        bboxes_yolo  = np.clip(bboxes_yolo, 0, 1)
        all_bboxes.extend(bboxes_yolo)
        for bbox_idx in range(len(bboxes_yolo)):
            annot = [str(labels[bbox_idx])]+ list(bboxes_yolo[bbox_idx].astype(str))+(['\n'] if num_bbox!=(bbox_idx+1) else [''])
            annot = ' '.join(annot)
            annot = annot.strip(' ')
            f.write(annot)
            
print('Missing:',cnt)

  0%|          | 0/4919 [00:00<?, ?it/s]

Missing: 0


### Configuration

In [16]:
%%time
import yaml

cwd = '/content/'

# case that use .2 split data 
#train_only_df = train_df[train_df.is_train==True]
#valid_only_df = train_df[train_df.is_train==False]

# case that use fold5 split data 
train_only_df = train_df[train_df.fold!=FOLD]
valid_only_df = train_df[train_df.fold==FOLD]

with open(os.path.join(cwd, 'train.txt'), 'w') as f:
    for path in train_only_df.image_path.tolist():
        f.write(path+'\n')

with open(os.path.join(cwd, 'val.txt'), 'w') as f:
    for path in valid_only_df.image_path.tolist():
        f.write(path+'\n')

data = dict(
    path = '/content/',
    train = os.path.join(cwd, 'train.txt'),
    val = os.path.join(cwd, 'val.txt'),
    nc = 1,
    names = ['cots'],
)

with open(os.path.join(cwd, 'cots.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join(cwd, 'cots.yaml'), 'r')

print('\nyaml')
print(f.read())


yaml
names:
- cots
nc: 1
path: /content/
train: /content/train.txt
val: /content/val.txt

CPU times: user 25.8 ms, sys: 4.04 ms, total: 29.8 ms
Wall time: 97.3 ms


## hyper parameter

In [17]:
data = dict(
    lr0 = 0.01,  # initial learning rate (SGD=1E-2, Adam=1E-3)
    lrf = 0.1,  # final OneCycleLR learning rate (lr0 * lrf)
    momentum = 0.937,  # SGD momentum/Adam beta1
    weight_decay = 0.0005,  # optimizer weight decay 5e-4
    warmup_epochs = 2.0,  # warmup epochs (fractions ok)
    warmup_momentum = 0.8,  # warmup initial momentum
    warmup_bias_lr = 0.1,  # warmup initial bias lr
    box = 0.05,  # box loss gain
    cls = 0.3,  # cls loss gain
    cls_pw = 1.0,  # cls BCELoss positive_weight
    obj = 0.7,  # obj loss gain (scale with pixels)
    obj_pw = 1.0,  # obj BCELoss positive_weight
    iou_t = 0.20,  # IoU training threshold
    anchor_t = 4.0,  # anchor-multiple threshold
    # anchors = 3,  # anchors per output layer (0 to ignore)
    fl_gamma = 0.0,  # focal loss gamma (efficientDet default gamma=1.5)
    hsv_h = 0.015,  # image HSV-Hue augmentation (fraction)
    hsv_s = 0.7,  # image HSV-Saturation augmentation (fraction)
    hsv_v = 0.4,  # image HSV-Value augmentation (fraction)
    degrees = 0.0,  # image rotation (+/- deg)
    translate = 0.1,  # image translation (+/- fraction)
    scale = 0.9,  # image scale (+/- gain)
    shear = 0.0,  # image shear (+/- deg)
    perspective = 0.0,  # image perspective (+/- fraction), range 0-0.001
    flipud = 0.5,  # image flip up-down (probability)
    fliplr = 0.5,  # image flip left-right (probability)
    mosaic = 1.0,  # image mosaic (probability)
    mixup = 0.6,  # image mixup (probability)
    copy_paste = 0.4,  # segment copy-paste (probability)
)

with open(os.path.join(cwd, 'my_hyp.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join(cwd, 'my_hyp.yaml'), 'r')

print('\nyaml')
print(f.read())


yaml
anchor_t: 4.0
box: 0.05
cls: 0.3
cls_pw: 1.0
copy_paste: 0.4
degrees: 0.0
fl_gamma: 0.0
fliplr: 0.5
flipud: 0.5
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
iou_t: 0.2
lr0: 0.01
lrf: 0.1
mixup: 0.6
momentum: 0.937
mosaic: 1.0
obj: 0.7
obj_pw: 1.0
perspective: 0.0
scale: 0.9
shear: 0.0
translate: 0.1
warmup_bias_lr: 0.1
warmup_epochs: 2.0
warmup_momentum: 0.8
weight_decay: 0.0005



## Training

In [18]:
%cd yolov5

from yolov5 import utils
display = utils.notebook_init()  # check

YOLOv5 🚀 v6.0-159-gdb6ec66 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (4 CPUs, 25.5 GB RAM, 73.5/166.8 GB disk)


In [19]:
!python train.py --img 2624 \
--batch 4 \
--epochs 20 \
--data /content/cots.yaml \
--hyp /content/my_hyp.yaml \
--weights yolov5s6.pt \
--patience=5 \
--workers 0

train: weights=yolov5s6.pt, cfg=, data=/content/cots.yaml, hyp=/content/my_hyp.yaml, epochs=20, batch_size=4, imgsz=2624, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=0, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=5, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 415, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 415 (delta 173), reused 173 (delta 173), pack-reused 240
Receiving objects: 100% (415/415), 304.73 KiB | 12.70 MiB/s, done.
Resolving deltas: 100% (289/289), completed with 54 local objects.
From https://github.com/ultralytics/yolov5
   db6ec66..856d4e5  master              -> origin/master

#### remove val folder

In [ ]:
!rm -r /content/yolov5/runs/val

In [ ]:
def calc_f2(R, P):
    return (5 * P * R) / ((4*P) + R)

R = 0.616
P = 0.844

F2_score = calc_f2(R, P)
print(F2_score)

0.6511823647294588


### Calculate F2 score on validation set

In [ ]:
val_conf_thres = 0.20
val_iou_thres = 0.30

In [ ]:
import val
# /content/yolov5/runs/train/exp/weights/best.pt
# /content/drive/MyDrive/Colab/Great-Barrier-Reef/model/yolov5/ver12/exp/weights/best.pt
!python val.py --data /content/cots.yaml \
    --weights /content/yolov5/runs/train/exp/weights/best.pt \
    --imgsz 3200 \
    --conf-thres 0.20 \
    --iou-thres 0.30 \
    --save-txt \
    --save-conf \
    --exist-ok

val: data=/content/cots.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=3200, conf_thres=0.2, iou_thres=0.3, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=runs/val, name=exp, exist_ok=True, half=False, dnn=False
YOLOv5 🚀 v6.0-159-gdb6ec66 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 378 layers, 35248920 parameters, 0 gradients, 49.0 GFLOPs
val: Scanning '/content/val.cache' images and labels... 773 found, 0 missing, 0 empty, 0 corrupted: 100% 773/773 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5  mAP@.3:.8:   0% 0/25 [00:08<?, ?it/s]
Traceback (most recent call last):
  File "val.py", line 372, in <module>
    main(opt)
  File "val.py", line 345, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py"

#### Check how predicted bounding box is created

In [ ]:
# val bbox result directory
PRD_BBOX_DIR = '/content/yolov5/runs/val/exp/labels/'
print(f'made bounding box of {len(os.listdir(PRD_BBOX_DIR))} images in validation set ')

made bounding box of 0 images in validation set 


In [ ]:
val_images = []
with open('/content/val.txt', 'r') as f:
    while True:
        r = f.readline().rstrip()
        if not r:
            break
        val_images.append(os.path.basename(r))
print(f'{len(val_images)} image in validation set')

773 image in validation set


In [ ]:
not_processed_images = val_images.copy()
for file in os.listdir(PRD_BBOX_DIR):
    img_name = file[:-4]+'.jpg'
    if img_name in val_images:
        not_processed_images.remove(img_name)
print(f"yolov5 model doesn't create bounding box for {len(not_processed_images)} images")

yolov5 model doesn't create bounding box for 773 images


In [ ]:
# model didn't detect starfish in "not_processed_images" - it will be calculated as False Negative(FN)
# run code to know that there exist ground truth bounding boxs in "not_processed_images"
# in fact, /kaggle/images/ only include images which have bounding boxs
for image_name in not_processed_images[:20]:
    img = cv2.imread('/content/dataset/images/'+image_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #plt.imshow(img)
    #plt.title(image_name)
    #plt.show()
    txt_name = image_name[:-4]+'.txt'
    with open('/content/dataset/labels/'+txt_name, 'r') as f:
        r = f.read()
        count = r.count('\n')+1
        print(f"{image_name} has {count} ground truth bounding box exits")

0-16.jpg has 1 ground truth bounding box exits
0-17.jpg has 1 ground truth bounding box exits
0-18.jpg has 1 ground truth bounding box exits
0-19.jpg has 1 ground truth bounding box exits
0-20.jpg has 1 ground truth bounding box exits
0-21.jpg has 1 ground truth bounding box exits
0-22.jpg has 1 ground truth bounding box exits
0-23.jpg has 1 ground truth bounding box exits
0-24.jpg has 1 ground truth bounding box exits
0-25.jpg has 1 ground truth bounding box exits
0-26.jpg has 1 ground truth bounding box exits
0-27.jpg has 1 ground truth bounding box exits
0-28.jpg has 1 ground truth bounding box exits
0-29.jpg has 1 ground truth bounding box exits
0-30.jpg has 1 ground truth bounding box exits
0-31.jpg has 1 ground truth bounding box exits
0-32.jpg has 1 ground truth bounding box exits
0-33.jpg has 1 ground truth bounding box exits
0-34.jpg has 1 ground truth bounding box exits
0-35.jpg has 2 ground truth bounding box exits


In [ ]:
def calc_iou(bboxes1, bboxes2, bbox_mode='xywh'):
    assert len(bboxes1.shape) == 2 and bboxes1.shape[1] == 4
    assert len(bboxes2.shape) == 2 and bboxes2.shape[1] == 4
    
    bboxes1 = bboxes1.copy()
    bboxes2 = bboxes2.copy()
    
    if bbox_mode == 'xywh':
        bboxes1[:, 2:] += bboxes1[:, :2]
        bboxes2[:, 2:] += bboxes2[:, :2]

    x11, y11, x12, y12 = np.split(bboxes1, 4, axis=1)
    x21, y21, x22, y22 = np.split(bboxes2, 4, axis=1)
    xA = np.maximum(x11, np.transpose(x21))
    yA = np.maximum(y11, np.transpose(y21))
    xB = np.minimum(x12, np.transpose(x22))
    yB = np.minimum(y12, np.transpose(y22))
    interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)
    return iou

def f_beta(tp, fp, fn, beta=2):
    return (1+beta**2)*tp / ((1+beta**2)*tp + beta**2*fn+fp)

def calc_is_correct_at_iou_th(gt_bboxes, pred_bboxes, iou_th, verbose=False):
    gt_bboxes = gt_bboxes.copy()
    pred_bboxes = pred_bboxes.copy()
    
    tp = 0
    fp = 0
    for k, pred_bbox in enumerate(pred_bboxes): # fixed in ver.7
        ious = calc_iou(gt_bboxes, pred_bbox[None, 1:])
        max_iou = ious.max()
        if max_iou > iou_th:
            tp += 1
            gt_bboxes = np.delete(gt_bboxes, ious.argmax(), axis=0)
        else:
            fp += 1
        if len(gt_bboxes) == 0:
            fp += len(pred_bboxes) - (k + 1) # fix in ver.7
            break

    fn = len(gt_bboxes)
    return tp, fp, fn

def calc_is_correct(gt_bboxes, pred_bboxes):
    """
    gt_bboxes: (N, 4) np.array in xywh format
    pred_bboxes: (N, 5) np.array in conf+xywh format
    """
    if len(gt_bboxes) == 0 and len(pred_bboxes) == 0:
        tps, fps, fns = 0, 0, 0
        return tps, fps, fns
    
    elif len(gt_bboxes) == 0:
        tps, fps, fns = 0, len(pred_bboxes)*11, 0
        return tps, fps, fns
    
    elif len(pred_bboxes) == 0:
        tps, fps, fns = 0, 0, len(gt_bboxes)*11
        return tps, fps, fns
    
    pred_bboxes = pred_bboxes[pred_bboxes[:,0].argsort()[::-1]] # sort by conf
    
    tps, fps, fns = 0, 0, 0
    for iou_th in np.arange(0.3, 0.85, 0.05):
        tp, fp, fn = calc_is_correct_at_iou_th(gt_bboxes, pred_bboxes, iou_th)
        tps += tp
        fps += fp
        fns += fn
    return tps, fps, fns

def calc_f2_score(gt_bboxes_list, pred_bboxes_list, verbose=False):
    """
    gt_bboxes_list: list of (N, 4) np.array in xywh format
    pred_bboxes_list: list of (N, 5) np.array in conf+xywh format
    """
    tps, fps, fns = 0, 0, 0
    for gt_bboxes, pred_bboxes in zip(gt_bboxes_list, pred_bboxes_list):
        tp, fp, fn = calc_is_correct(gt_bboxes, pred_bboxes)
        tps += tp
        fps += fp
        fns += fn
        if verbose:
            num_gt = len(gt_bboxes)
            num_pred = len(pred_bboxes)
            print(f'num_gt:{num_gt:<3} num_pred:{num_pred:<3} tp:{tp:<3} fp:{fp:<3} fn:{fn:<3}')
    return f_beta(tps, fps, fns, beta=2)

In [ ]:
gt_bboxs_list, prd_bboxs_list = [], []
count = 0
for image_file in val_images:
    txt_name = image_file[:-4]+'.txt'
    gt_bboxs = []
    prd_bboxs = []
    with open('/content/'+LABEL_DIR+'/'+txt_name, 'r') as f:
        while True:
            r = f.readline().rstrip()
            if not r:
                break
            r = r.split()[1:]
            bbox = np.array(list(map(float, r)))
            gt_bboxs.append(bbox)
    if os.path.exists(PRD_BBOX_DIR+txt_name):
        with open(PRD_BBOX_DIR+txt_name, 'r') as f:
            while True:
                r = f.readline().rstrip()
                if not r:
                    break
                r = r.split()[1:]
                r = [r[4], *r[:4]]
                bbox = np.array(list(map(float, r)))
                prd_bboxs.append(bbox)
    gt_bboxs, prd_bboxs = np.array(gt_bboxs), np.array(prd_bboxs)
    gt_bboxs_list.append(gt_bboxs)
    prd_bboxs_list.append(prd_bboxs)
    count += 1
print(f'{count} bound boxs appended to list')

773 bound boxs appended to list


In [ ]:
score = calc_f2_score(gt_bboxs_list, prd_bboxs_list, verbose=True)

num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:1   num_pred:0   tp:0   fp:0   fn:11 
num_gt:2   num_pred:0   tp:0   fp:0   fn:22 
num_gt:2   num_pred:0   tp:0   fp:0   fn:22 
num_gt:2   num_pred:0   tp:0   fp:0   fn:22 
num_gt:2  

In [ ]:
print('val_conf_thres & val_iou_thres is ', val_conf_thres, val_iou_thres)
print(f'f2 score for validation set is {score}')

val_conf_thres & val_iou_thres is  0.2 0.3
f2 score for validation set is 0.0


### Copy result

In [20]:
!cp -r /content/yolov5/runs/train/exp /content/drive/MyDrive/Colab/Great-Barrier-Reef/model/exp